# Data and supporting files Setup

In [ ]:
# importing pandas, geopy, math
# reading and saving csv files of the downloaded resale flat prices into dataframes

import pandas as pd
from geopy.geocoders import Nominatim
from math import cos, asin, sqrt, pi

df_1990 = pd.read_csv('resale_1990_1999.csv')
df_2000 = pd.read_csv('resale_2000_2012.csv')
df_2012 = pd.read_csv('resale_2012_2014.csv')
df_2015 = pd.read_csv('resale_2015_2016.csv')
df_2017 = pd.read_csv('resale_2017_2021.csv')

# calculating the remaining lease for all the resale flats even for the data sets which did not initially have

for n in [df_1990, df_2000, df_2012, df_2015, df_2017]:
    n['remaining_lease'] = 99 - n.month.str[:4].astype(int) + n.lease_commence_date   


In [720]:
# loading supporting files - cpi, mrt station coords, mall coords
df_cpi = pd.read_csv('CPI.csv')
df_mrt = pd.read_csv('mrtsg.csv')
df_mrt = df_mrt[df_mrt['COLOR'] != 'OTHERS']
df_mall = pd.read_csv('mallsg.csv')

In [285]:
# appending all the data sets together to form one data set with continuous years

df_all = df_1990
for n in [df_2000, df_2012, df_2015, df_2017]:
    df_all = df_all.append(n)

    
# synthesising new data based on the exisitng ones

df_all['max_storey'] = df_all.storey_range.str[-2::].astype(int)
df_all['year'] = df.month.str[:4].astype(int)

# Setting up functions to find location (coordinates) and distance

In [264]:
# set up function to calculate the distance between two locations based on their coordinates
def distance(row):
    lat2 = float(row['Latitude'])
    lon2 = float(row['Longitude'])
    lat1 = float(row['lat'])
    lon1 = float(row['lon'])
    
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...

In [201]:
def find_location(row):
    # function to find the latitude and longiitude when given the name of the place
    
    place = row['place']
    
    location = geolocator.geocode(place)
    
    if location != None:
        return location.address, location.latitude, location.longitude, location.raw['importance']
    else:
        return "Not Found", "Not Found", "Not Found", "Not Found"

# Using Geopy to find the coordinates of the resale flats

In [281]:
# using individual geolocating steps for the coordinates that are not found based on the saved names in the original data set
geolocator = Nominatim(user_agent="http", timeout=2)
location = geolocator.geocode("Ranggung Singapore")
new = [location.address, location.latitude, location.longitude, location.raw['importance']]
new

['Ranggung, Sengkang East Avenue, Seng Kang, Northeast, 544207, Singapore',
 1.3840474,
 103.8974006,
 0.40463416789671997]

In [222]:
# parsing the data so that the the street names can be serached up easily using geopy
''' df_street_name = pd.DataFrame(df_all.street_name.unique())
df_street_name.columns = ['street_name']
df_street_name['place'] = (df_street_name['street_name'] + ' Singapore').astype(str)
df_street_name["place"]= df_street_name["place"].str.replace("upp", "upper", case = False)
df_street_name["place"]= df_street_name["place"].str.replace("BT", "bukit", case = False)
df_street_name["place"]= df_street_name["place"].str.replace("nth", "north", case = False)
df_street_name["place"]= df_street_name["place"].str.replace("ctrl", "central", case = False)
df_street_name["place"]= df_street_name["place"].str.replace("lor", "lorong", case = False)
df_street_name["place"]= df_street_name["place"].str.replace("kg", "kampong", case = False)
df_street_name["place"]= df_street_name["place"].str.replace("jln", "jalan", case = False)
df_street_name["place"]= df_street_name["place"].str.replace("tg", "tanjong", case = False)

df_street_name '''

SyntaxError: invalid syntax (<ipython-input-222-5c2b4d8867bb>, line 1)

In [318]:
# using the find location function on the street name dataframe and saving the resulting dataframe into a csv file
''' df_street_name[['address', 'lat', 'lon', 'importance']] = df_street_name.apply(find_location, axis="columns", result_type="expand")
df_street_name.to_csv('street_name_coords.csv') '''

' df_street_name[[\'address\', \'lat\', \'lon\', \'importance\']] = df_street_name.apply(find_location, axis="columns", result_type="expand")\ndf_street_name '

In [339]:
# final touches done on some of the addresses hence read once again
df_address = pd.read_csv('street_name_coords_v1.csv')

# Finding and recording the distance from the nearest mrt

In [346]:
df_address['key'] = 0
df_mrt['key'] = 0
df_union = df_address.merge(df_mrt, on='key', how='outer')

In [302]:
# performing the distance calculation between street_name and mrt stations
df_union[['distance']] = df_union.apply(distance, axis="columns", result_type="expand")

In [398]:
# getting the distance from the nearest MRT for each street name and removing the additional column header
df_nearest = df_union.groupby('street_name').agg({'distance': ['min']})
df_nearest = df_nearest['distance']
df_nearest = df_nearest.rename(columns={'min': 'dist_mrt'})

In [399]:
# set the df_all index to street_name to combine with df_nearest to get the dist_mrt
df_all.reset_index(inplace=True)
df_all.set_index('street_name', inplace = True)
df_final = pd.concat([df_all, df_nearest], axis=1, join="inner")

# Finding the distance from CBD (downtown core)

In [360]:
# setting up the necessary dist from cbd column and the lat and lon of singapore's downtown core
df_union2 = df_address
df_union2['dist_cbd'] = 0
df_union2['Latitude'], df_union2['Longitude'] = [1.287953, 103.851784] # The latitude of Downtown Core, Singapore is 1.287953, and the longitude is 103.851784.

In [361]:
# performing the distance calculation between street_name and singapore's downtown core
df_union2[['dist_cbd']] = df_union2.apply(distance, axis="columns", result_type="expand")

In [696]:
# adding the dist from cbd to the final data set and saving only the required features to the dataset
df_distcbd = df_union2[['street_name','dist_cbd']]
df_distcbd.set_index('street_name')
df_final2 = pd.concat([df_final, df_distcbd.set_index('street_name')], axis=1, join="inner")
df_dataset = df_final2[['year','resale_price', 'town', 'flat_type', 'floor_area_sqm', 'flat_model', 'remaining_lease',
                       'max_storey', 'dist_mrt', 'dist_cbd']]
df_dataset = df_dataset.reset_index()

# Merging the cpi data to the final dataset

In [697]:
df_cpi2 = df_cpi.loc[df_cpi['level_1'] == 'All Items'][['year','value']]
df_cpi2.reset_index(drop = True)
df_cpi2.value = df_cpi2.value.astype('float')

# creating a dictionary out of the CPI dataframe
year = list(df_cpi2.year)
cpi = list(df_cpi2.value)
cpi_dict = {year[i]: cpi[i] for i in range(len(year))}
cpi_dict[2021] = 101.5 # set cpi data of 2021 to be an average of the YTD cpi data

# joining the cpi data into the final dataset dataframe
df_dataset.reset_index()
df_dataset['cpi'] = df_dataset['year']
df_dataset = df_dataset.replace({"cpi": cpi_dict})
#df_dataset.set_index('resale_price', inplace = True)

# Changing categorical data into numerical equivalents

In [753]:
# for the purposes of modelling, the towns need to be converted from categorical data into continuous/numerical data
# for town: the median resale price for each town and finding its premium as compared to the median resale price
df_town = df_dataset.groupby(['town'],as_index=False).median()[['town','resale_price']]
df_town['town_premium'] = df_town['resale_price']-df_town['resale_price'].median()
df_datafinal = pd.merge(df_dataset, df_town[['town','town_premium']],on='town')

# for flat model: the median resale price for each town and finding its premium as compared to the median resale price
df_flat_model = df_dataset.groupby(['flat_model'],as_index = False).median()[['flat_model','resale_price']]
df_flat_model['model_premium'] = df_flat_model['resale_price']-df_flat_model['resale_price'].median()
df_datafinal = pd.merge(df_datafinal, df_flat_model[['flat_model','model_premium']],on='flat_model')

# Finding nearest mall and including nearest distance

In [755]:
# tidying up the mall dataframe
df_mall.rename(columns={"latitude": "Latitude", "longitude": "Longitude"})
df_mall['key'] = 0
df_mall_address = df_address.merge(df_mall.drop(columns='Unnamed: 0'), on='key', how='outer')

# performing the distance calculation between street_name and mrt stations
df_mall_address[['distance']] = df_mall_address.apply(distance, axis="columns", result_type="expand")

# getting the distance from the nearest MRT for each street name and removing the additional column header
df_nearest_mall = df_mall_address.groupby('street_name').agg({'distance': ['min']})
df_nearest_mall = df_nearest_mall['distance']
df_nearest_mall = df_nearest_mall.rename(columns={'min': 'dist_mall'})
df_datafinal = pd.merge(df_datafinal, df_nearest_mall.reset_index()[['street_name','dist_mall']],on='street_name')

In [757]:
#saving a copy of the final data set for the next part of analysis
df_datafinal.set_index('resale_price', inplace = True)
df_datafinal.to_csv('final_dataset.csv')